In [1]:
# 多输入通道
import gluonbook as gb
from mxnet import nd

In [4]:
def corr2d_multi_in(X, K):
    return nd.add_n(*[gb.corr2d(x, k) for x, k in zip(X, K)])

In [5]:
X = nd.array([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
        [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = nd.array([[[0, 1], [2, 3]], [[1, 2], [3, 4]]])
corr2d_multi_in(X, K)


[[  56.   72.]
 [ 104.  120.]]
<NDArray 2x2 @cpu(0)>

In [6]:
[gb.corr2d(x, k) for x, k in zip(X, K)]

[
 [[ 19.  25.]
  [ 37.  43.]]
 <NDArray 2x2 @cpu(0)>, 
 [[ 37.  47.]
  [ 67.  77.]]
 <NDArray 2x2 @cpu(0)>]

In [7]:
# 多输出通道
def corr2d_multi_in_out(X, K):
    # 对 K 的第 0 维遍历，每次同输⼊ X 做互相关计算。所有结果使⽤ stack 函数合并在⼀起。
    return nd.stack(*[corr2d_multi_in(X, k) for k in K])

In [8]:
K.shape

(2, 2, 2)

In [9]:
K+1


[[[ 1.  2.]
  [ 3.  4.]]

 [[ 2.  3.]
  [ 4.  5.]]]
<NDArray 2x2x2 @cpu(0)>

In [10]:
K = nd.stack(K, K + 1, K + 2)

In [11]:
K.shape

(3, 2, 2, 2)

In [12]:
corr2d_multi_in_out(X, K)


[[[  56.   72.]
  [ 104.  120.]]

 [[  76.  100.]
  [ 148.  172.]]

 [[  96.  128.]
  [ 192.  224.]]]
<NDArray 3x2x2 @cpu(0)>

In [13]:
# 1*1 卷积层
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = nd.dot(K, X)  # 全连接层的矩阵乘法
    return Y.reshape((c_o, h, w))

In [14]:
X = nd.random.uniform(shape=(3, 3, 3))
K = nd.random.uniform(shape=(2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
(Y1 - Y2).norm().asscalar() < 1e-6

True

In [ ]:
# 1 × 1 卷积失去了卷积层可以识别⾼和宽维度上相邻元素构成的模式的功能。
# 实际上， 1 × 1 卷积的主要计算发⽣在通道维上。
# 1 × 1 卷积层被当做保持⾼和宽维度形状不变的全连接层使⽤。
# 我们可以通过调整⽹络层之间的通道数来控制模型复杂度。

# 假设我们将通道维当做是特征维，将⾼和宽维度上的元素当成数据样本，那么 1 × 1 卷积层
# 的作⽤与全连接层等价。

# 使⽤多通道可以拓展卷积层的模型参数。